In [1]:
import pandas as pd
import quandl, math 
import numpy as np
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import svm

In [2]:
df = quandl.get('WIKI/GOOGL')

In [3]:
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.00 
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.00 

df = df[['Adj. Close','HL_PCT','PCT_change','Adj. Volume']]

In [4]:
forecast_col = 'Adj. Close'
df.fillna(-99999,inplace=True)

In [5]:
forecast_out = int(math.ceil(0.01 * len(df)))

In [6]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [7]:
X = np.array(df.drop('label',1))

X = scale(X)

X = X[:-forecast_out]
X_lately = X[-forecast_out:]

df.dropna(inplace=True)
y = np.array(df['label'])
y = np.array(df['label'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
clf = LinearRegression()
clf.fit(X_train,y_train)

accuracy = clf.score(X_test, y_test)

In [16]:
clf = svm.SVR()
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

0.7872924468448045

In [17]:
forecast_set = clf.predict(X_lately)
print(forecast_set,accuracy,forecast_out)

[691.86524467 669.79072314 657.87849482 679.32071041 670.5801329
 681.75066209 681.54730879 680.5142502  679.65261666 680.7288312
 693.33193632 597.69336314 614.84822973 675.12906438 657.55935675
 664.8348269  656.32519051 661.1315372  660.95221807 574.11400177
 597.22046378 635.39201298 647.6750923  630.92141629 569.28473358
 620.41990311 579.78746916 620.98314416 606.50485352 617.62818313
 626.02747958 622.21191092 622.55961567 621.02745604 382.75425295] 0.9769102628428439 35


In [32]:
df['forecast'] = np.nan

last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day